# Tutorial outline
- Making a simple scatter plot 
    - Defining the 'marker' properties
- Setting the 'layout' of a plotly figure
    - Axes properties
    - templates
- Legend
- Adding annotations

- Making subplots
    - Default subplots
    - go.Splom
- Animations
    - Defining 'frames'
    - Defining the 'slider'
    - Play/pause buttons
- 3D figures
    - The 'scene' property of 'layout'
- Using update menus
    - Buttons
    - Dropdown menus

- Saving figures as html 
    - Opening in a browser
    - Pushing to Github
    - Hosting on websites (Github pages / Google Sites)
    - Hosting within Notion

In [1]:
# Download dependencies
!pip install astropy astroquery plotly dustmaps

DEPRECATION: Loading egg at /Users/cam/opt/miniconda3/envs/p311/lib/python3.11/site-packages/healsparse-1.8.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/cam/opt/miniconda3/envs/p311/lib/python3.11/site-packages/cartosky-0.1+45.g5d14bc5-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at /Users/cam/opt/miniconda3/envs/p311/lib/python3.11/site-packages/ephem-4.1.5-py3.11-macosx-10.9-x86_64.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
ERROR: Could not find a version that sa

In [2]:
# Required packages
import numpy as np
import pandas as pd
from astropy.coordinates import SkyCoord
import astropy.units as u
from astropy.io import fits
from astroquery.vizier import Vizier
import plotly.graph_objects as go
import plotly.express as px
import plotly.figure_factory as ff
import copy
from dustmaps.edenhofer2023 import Edenhofer2023Query
from dustmaps.leike2020 import Leike2020Query

#plot_save_directory = './html_plots/'
plot_save_directory = '/Users/cam/Desktop/astro_research/radcliffe/cam_website_clone/cam_website/orion_plots/' # <-- TODO: change this

### Note
If you are running in Google Colab -- In order to save the plots, create a new folder (menu on the left) and then set above `plot_save_directory = ./folder_name`

# Load in Data

We will be visualizing two different datasets for the region of Orion. One are the young stars that have formed in Orion as part of ~20 different sub-groupings (clusters). Each star contains Gaia-measured sky coordinates (ra, dec), parallaxes (distances), proper motions (pmra, pmdec), and sometimes radial velocities. We can convert all of these stars to Galactic cartesian coordinates (x,y,z).

The other dataset we will visualize is a reconcstruction of the 3D dust distribution in Orion. This specific 3D dust map is from Leike et al. 2020. 

In [3]:
# Query dustmap (if needed)

# Only run if leike2020 is not already downloaded
import dustmaps.leike2020
dustmaps.leike2020.fetch() # <-- NOTE: Comment this line if already ran onc

File "mean_std.h5" appears to exist already. Call `fetch(clobber=True)` to force overwriting of existing file.


In [4]:
# --------------- Cell block not relevant to Plotly -----------------


# Query dust in region of Orion

x = np.arange(-450, -202, 5)
y = np.arange(-260, -42, 5)
z = np.arange(-250, 2, 5)

x_bounds = [x.min(), x.max()]
y_bounds = [y.min(), y.max()]
z_bounds = [z.min(), z.max()]

xx, yy, zz = np.meshgrid(x, y, z)

sc = SkyCoord(
    xx.flatten() * u.pc,
    yy.flatten() * u.pc,
    zz.flatten() * u.pc,
    frame='galactic',
    representation_type = 'cartesian'
)

# query the dust map
#dust = Edenhofer2023Query().query(sc)
dust_cart = Leike2020Query().query(sc)


# Querey on sky
ra = np.arange(70, 100, 0.1)
dec = np.arange(-20, 20, 0.1)
distance = np.arange(350, 400, 5)

ra_mesh, dec_mesh, distance_mesh = np.meshgrid(ra, dec, distance)
sc_sky = SkyCoord(
    ra = ra_mesh,
    dec = dec_mesh,
    distance = distance_mesh,
    frame='icrs',
    unit=(u.deg, u.deg, u.pc)
)
dust_icrs = Leike2020Query().query(sc_sky)
dust_2d = np.nansum(dust_icrs, axis=2)

In [5]:
# --------------- Cell block not relevant to Plotly -----------------

# Query the stellar cluster data
orion_catalog_id = 'J/ApJ/917/21/'
v = Vizier(row_limit=-1)
result = v.get_catalogs(orion_catalog_id)

df_clusters = result[0].to_pandas()
df_stars = result[1].to_pandas()

df_stars = pd.merge(left=df_stars, right=df_clusters, left_on='Group', right_on='Group', how = 'left', suffixes=('', '_cluster'))

In [6]:
# --------------- Cell block not relevant to Plotly -----------------

# Convert star/cluster data to cartesian
# ------------- Clusters -------------
sc = SkyCoord(
    ra = df_clusters['RAdeg'].values * u.deg,
    dec = df_clusters['DEdeg'].values * u.deg,
    distance = df_clusters['Dist'].values*u.pc,
    pm_ra_cosdec = df_clusters['pmRA'].values * u.mas/u.yr,
    pm_dec = df_clusters['pmDE'].values * u.mas/u.yr,
    radial_velocity = df_clusters['HRV'].values * u.km/u.s,
    frame='icrs'
)
sc = sc.transform_to('galactic')
sc.representation_type = 'cartesian'
sc.differential_type = 'cartesian'

df_clusters['x_helio'] = sc.u.value
df_clusters['y_helio'] = sc.v.value
df_clusters['z_helio'] = sc.w.value
df_clusters['U'] = sc.U.value
df_clusters['V'] = sc.V.value
df_clusters['W'] = sc.W.value


# ------------- Stars -------------
df_stars['Combined_RV'] = df_stars[['RVel2', 'RVel3', 'RVel1']].bfill(axis=1).iloc[:, 0]
sc = SkyCoord(
    ra = df_stars['RA_ICRS'].values * u.deg,
    dec = df_stars['DE_ICRS'].values * u.deg,
    distance = (1000/df_stars['Plx']).values*u.pc,
    pm_ra_cosdec = df_stars['pmRA'].values * u.mas/u.yr,
    pm_dec = df_stars['pmDE'].values * u.mas/u.yr,
    radial_velocity = df_stars['Combined_RV'].values * u.km/u.s,
    frame='icrs'
)
sc = sc.transform_to('galactic')
sc.representation_type = 'cartesian'
sc.differential_type = 'cartesian'

df_stars['x_helio'] = sc.u.value
df_stars['y_helio'] = sc.v.value
df_stars['z_helio'] = sc.w.value
df_stars['U'] = sc.U.value
df_stars['V'] = sc.V.value
df_stars['W'] = sc.W.value




# Visualize a star-forming region with Plotly (2D)

### Sky View of stars: 
Let's first visualize the famous stars of Orion

In [7]:
# Star coordinates and names
ra_orion_famous = [88.6, 81.2, 83.03, 84.0, 85.2, 78.6, 86.9, 83.7]
dec_orion_famous = [7.2, 6.3, -0.4, -1.22, -1.9, -8.1, -9.7, 9.8]
names = ['Betelgeuse', 'Bellatrix', 'Mintaka', 'Alnilam', 'Alnitak', 'Rigel', 'Saiph', 'Meissa']
star_colors = ['orange', 'cyan', 'cyan', 'cyan', 'cyan', 'cyan', 'cyan', 'cyan']

### go.Scatter

In [8]:
scatter_famous_stars = go.Scatter(
    x = ra_orion_famous, 
    y = dec_orion_famous, 
    mode='markers', 
    marker=dict(
        size=10, 
        color=star_colors), 

    name = 'Famous stars',
    text=names) # Setting the marker properties

fig_ra_dec = go.Figure(scatter_famous_stars)
fig_ra_dec.write_html(plot_save_directory + 'famous_stars.html')
#fig.show()

### go.Layout
The above plot doesn't look amazing, so let's format the plot.

In [9]:
# setting the layout
layout_ra_dec = go.Layout(
    template = 'plotly_dark',
    title = 'Stars in Orion',
    xaxis = dict(
        title = 'RA', 
        range = [100, 70]
        ),
    yaxis = dict(
        title = 'DEC', 
        range = [-20, 20], 
        scaleanchor = 'x', 
        scaleratio = 1),
    width = 700, 
    height = 700,
    # paper_bgcolor='black',
    # plot_bgcolor='black',
)
fig_ra_dec['layout'] = layout_ra_dec


# making annotations
annotations = []
for ra_star, dec_star, name_star in zip(ra_orion_famous, dec_orion_famous, names):
    annotations.append(dict(x=ra_star, y=dec_star+1, xref='x', yref='y', text=name_star, showarrow=False, arrowhead=2, ax=0, ay=-40))
layout_ra_dec['annotations'] = annotations
fig_ra_dec.update_layout(layout_ra_dec)
fig_ra_dec.write_html(plot_save_directory + 'famous_stars.html')

#fig.show()

### Including multiple traces in a figure (go.Scatter with go.Heatmap)

In [10]:
fig_ra_dec_b = go.Figure()

gaia_stars_scatter = go.Scatter(
    x = df_stars['RA_ICRS'],
    y = df_stars['DE_ICRS'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'], # <---- NOTE: color points by a variable
        colorscale = 'plasma',
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)

dust_image = go.Heatmap(
    z = dust_2d,
    x = ra,
    y = dec,
    colorscale = 'Gray',
    zmin = 0,
    zmax = 0.2,
    opacity=1.,
    showscale=False,
    showlegend=True,
    name = 'Dust Map'
)

fig_ra_dec_b.add_traces([gaia_stars_scatter, dust_image])
layout_ra_dec['annotations'] = []
fig_ra_dec_b.update_layout(layout_ra_dec)
fig_ra_dec_b.write_html(plot_save_directory + 'orion_ra_dec.html')

#fig_ra_dec.show()

### Proper motions diagram (another example of go.Scatter)

In [11]:
pm_scatter = go.Scatter(
    x = df_stars['pmRA'],
    y = df_stars['pmDE'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'],
        colorscale = 'Plasma',
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)

layout_pm = go.Layout(
    template = 'plotly_dark',
    paper_bgcolor='black',
    plot_bgcolor='black',
    title = 'Stars in Orion',
    xaxis = dict(
        title = 'pmRA', 
        range = [-5, 5]
        ),
    yaxis = dict(
        title = 'pmDE', 
        range = [-5, 5], 
        scaleanchor = 'x', 
        scaleratio = 1),
    width = 600, 
    height = 600
)

fig_pms = go.Figure(data=[pm_scatter], layout=layout_pm)
fig_pms.write_html(plot_save_directory + 'orion_proper_motions.html')
#fig.show()

### Making subplots

In [12]:
from plotly.subplots import make_subplots
fig_subplot = make_subplots(rows=1, cols=2, subplot_titles=('RA vs DEC', 'pmRA vs pmDEC'))



stars_scatter = go.Scatter(
    x = df_stars['RA_ICRS'],
    y = df_stars['DE_ICRS'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'],
        colorscale = 'plasma',
    ),
    selected = dict(
        marker = dict(
            color = 'red'
        )
    ),
    unselected=dict(
        marker = dict(
            color = 'blue'
        )
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)

pm_scatter = go.Scatter(
    x = df_stars['pmRA'],
    y = df_stars['pmDE'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'],
        colorscale = 'plasma',
    ),
    selected = dict(
        marker = dict(
            color = 'red'
        )
    ),
    unselected=dict(
        marker = dict(
            color = 'blue'
        )
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)
fig_subplot.add_trace(stars_scatter, row=1, col=1)
fig_subplot.add_trace(pm_scatter, row=1, col=2)


fig_subplot.update_layout(
    template = 'plotly_dark',
    paper_bgcolor='black',
    plot_bgcolor='black',
    title = 'Stars in Orion',
    width = 1200, 
    height = 600,
    xaxis = dict(
        title = 'RA', 
        range = [100, 70]
        ),
)

# update the axes
fig_subplot.update_xaxes(title_text='RA', range=[100, 70], row=1, col=1)
fig_subplot.update_yaxes(title_text='DEC', row=1, col=1)
fig_subplot.update_xaxes(title_text='pmRA', row=1, col=2)
fig_subplot.update_yaxes(title_text='pmDEC', row=1, col=2)

fig_subplot.write_html(plot_save_directory + 'orion_subplot.html')

### Making a scatter plot matrix (go.Splom)

In [13]:
splom_plot = go.Splom(
    dimensions=[
        dict(label='RA', values=df_stars['RA_ICRS']),
        dict(label='DEC', values=df_stars['DE_ICRS']),
        dict(label='Plx', values=df_stars['Plx']),
        dict(label='pmRA', values=df_stars['pmRA']),
        dict(label='pmDE', values=df_stars['pmDE'])
    ],
    marker=dict(
        color=df_stars['Age'],
        colorscale='plasma',
        opacity=1.,
        size=1
    ),
    selected = dict(
    marker = dict(
        opacity=1.,
        size = 3.
    )
    ),
    unselected=dict(
        marker = dict(
            opacity=.1,
        )
    ),
)

layout_splom = go.Layout(
    template = 'ggplot2',
    width = 800, 
    height = 800
)

fig_splom = go.Figure(data=[splom_plot], layout=layout_splom)
fig_splom.write_html(plot_save_directory + 'orion_scatter_matrix.html')
#fig_splom.show()

# Animations

### Making animations (go.Frame):
- go.Frame
- slider
- Play/pause buttons

In [14]:
# --------------- Cell block not relevant to Plotly -----------------
def move_stars(ra, dec, pmra, pmdec, time_steps):
    # mas/yr to deg/yr
    pmra = pmra / 3600000
    pmdec = pmdec / 3600000

    # get relative proper motions
    pmra_rel = pmra - np.median(pmra)
    pmdec_rel = pmdec - np.median(pmdec)

    # Initialize arrays to store the positions
    ra_positions = np.zeros((len(ra), len(time_steps)))
    dec_positions = np.zeros((len(dec), len(time_steps)))

    # Calculate the positions for each time step
    for i, time in enumerate(time_steps):
        # Calculate the positions
        ra_positions[:, i] = ra - pmra_rel * (time*1e6)
        dec_positions[:, i] = dec - pmdec_rel * (time*1e6)

    return ra_positions, dec_positions

In [15]:
# Defining the frames

# Obtain ra, dec positions as a function of time (using function defined in cell aboe)
df_stars_downsampled = df_stars.sample(3000)
time = np.arange(0, 30, 0.5) # units of Myr
ra_positions, dec_positions = move_stars(df_stars_downsampled['RA_ICRS'], df_stars_downsampled['DE_ICRS'], df_stars_downsampled['pmRA'], df_stars_downsampled['pmDE'],time)

frames = []
for i, t in enumerate(time):
    scatter_stars_t = go.Scatter(
        x = ra_positions[:, i],
        y = dec_positions[:, i],
        mode = 'markers',
        marker = dict(
            size = 2,
            opacity = 1.,
            color = df_stars_downsampled['Age'],
            colorscale = 'plasma',
        ),
    )
    frame = go.Frame(data=[scatter_stars_t], name=str(np.round(t,1)))
    frames.append(frame)


In [16]:
# setting properties of the slider
sliders = [dict(
    steps = [dict(
        args = [[f.name], dict(mode='immediate', frame=dict(duration=200, redraw=False))],
        label = str(f.name),
        method = 'animate',
    ) for f in frames],
    active = 0,
    transition = dict(duration=0),
    x = 0,
    y = 0,
)]

In [17]:
# setting properties of the play/pause buttons
play_pause = dict(
    type='buttons',
    showactive=False,
    buttons=[dict(label='Play',
                  method='animate',
                  args=[None, dict(frame=dict(duration=200, redraw=False), fromcurrent=True)]),
             dict(label='Pause',
                  method='animate',
                  args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate')])]
)

In [18]:
# putting them all together
layout_ra_dec_anim = copy.deepcopy(layout_ra_dec)
layout_ra_dec_anim['sliders'] = sliders
layout_ra_dec_anim['updatemenus'] = [play_pause]
layout_ra_dec_anim['annotations'] = None


data = frames[0]['data'] # NOTE: This step is important!


fig = go.Figure(data = data, frames=frames, layout=go.Layout(layout_ra_dec_anim))
fig.write_html(plot_save_directory + 'orion_stars_animation.html', auto_open=False, auto_play=False)

### Another animation (Stepping through the 3D dust cube)

In [19]:
dust_icrs.shape
frames_dust = []
for i in range(dust_icrs.shape[2]):
    dust_slice = dust_icrs[:, :, i]
    dust_image = go.Heatmap(
        z = dust_slice,
        x = ra,
        y = dec,
        colorscale = 'inferno',
        zmin = 0,
        zmax = 0.1,
        opacity=1.,
        showscale=False,
        showlegend=True,
        name = 'Dust Map'
    )
    frame = go.Frame(data=[dust_image], name=str(distance[i]))
    frames_dust.append(frame)

sliders = [dict(
    steps = [dict(
        args = [[f.name], dict(mode='immediate', frame=dict(duration=2, redraw=True))],
        label = str(f.name),
        method = 'animate',
    ) for f in frames_dust],
    active = 0,
    transition = dict(duration=0),
    x = 0,
    y = 0,
)]

play_pause = dict(
    type='buttons',
    showactive=False,
    buttons=[dict(label='Play',
                  method='animate',
                  args=[None, dict(frame=dict(duration=200, redraw=True), fromcurrent=True)]),
             dict(label='Pause',
                  method='animate',
                  args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate')])]
)
layout_ra_dec_dust_anim = copy.deepcopy(layout_ra_dec)
layout_ra_dec_dust_anim['sliders'] = sliders
layout_ra_dec_dust_anim['updatemenus'] = [play_pause]
layout_ra_dec_dust_anim['annotations'] = None
data = frames_dust[0]['data']
fig_dust = go.Figure(data = data, frames=frames_dust, layout=go.Layout(layout_ra_dec_dust_anim))
fig_dust.write_html(plot_save_directory + 'orion_dust_animation.html', auto_open=False, auto_play=False)

# 3D figures

In [20]:
gaia_stars_scatter3D = go.Scatter3d(
    x = df_stars['x_helio'],
    y = df_stars['y_helio'],
    z = df_stars['z_helio'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'],
        colorscale = 'plasma',
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)
sun_scatter3D = go.Scatter3d(
    x = [0],
    y = [0],
    z = [0],
    mode = 'markers',
    marker = dict(
        size = 5,
        color = 'yellow'
    ),
    name = 'Sun',
    hovertext = 'Sun'
)

layout_3d = go.Layout(
    template = 'plotly_dark',
    paper_bgcolor='black',
    plot_bgcolor='black',
    title = 'Stars in Orion',
    scene = dict(
        aspectmode = 'manual',
        aspectratio = dict(x = 1, y = 1, z = 1),
        xaxis = dict(title = 'x', range = [-400, 400]),
        yaxis = dict(title = 'y', range = [-400, 400]),
        zaxis = dict(title = 'z', range = [-400, 400]),
    ),
    width = 800, 
    height = 800)


fig_3d = go.Figure(data=[gaia_stars_scatter3D, sun_scatter3D], layout=layout_3d)
fig_3d.write_html(plot_save_directory + 'orion_3d.html')

### 3D Volume plots

In [21]:
volume_dust = go.Volume(
    x = xx.flatten(),
    y = yy.flatten(),
    z = zz.flatten(),
    value = dust_cart,
    flatshading=True,
    opacity=0.1,
    showscale=False,
    isomin=0.01,
    isomax=.1,
    colorscale=[[0, 'white'], [1., 'gray']],
    #colorscale = 'gray',
    # opacityscale='max',
    # reversescale=False,
    surface=dict(show=True,count=10),
    name = 'Dust Volume',
    showlegend=True,
    # spaceframe=dict(show=True),#,
    # contour=dict(show=False,width=4)
    )

layout_3d = go.Layout(
    template = 'plotly_dark',
    paper_bgcolor='black',
    plot_bgcolor='black',
    title = 'stars in orion',
    scene = dict(
        aspectmode = 'manual',
        aspectratio = dict(x = 1, y = 1, z = 1),
        xaxis = dict(title = 'x', range = [-400, 400]),
        yaxis = dict(title = 'y', range = [-400, 400]),
        zaxis = dict(title = 'z', range = [-400, 400]),
    ),
    width = 800, 
    height = 800)



fig_3d_dust = go.Figure(data=[gaia_stars_scatter3D, sun_scatter3D, volume_dust], layout=layout_3d)
fig_3d_dust.write_html(plot_save_directory + 'orion_3d_dust.html')

In [22]:
camera = dict(
    up=dict(x=0, y=0, z=1),
    center=dict(x=0, y=0, z=0),
    eye=dict(x=1.25, y=1.25, z=1.25)
)
# camera = dict(
#     eye=dict(x=0, y=0, z=2.5),
#     up=dict(x=0, y=1, z=0),
#     center=dict(x=0, y=0, z=0)
# )
scene = dict(
    aspectmode = 'manual',
    aspectratio = dict(x = 1, y = 1, z = 1),
    xaxis = dict(title = 'x', range = [np.min(x), np.max(x)]),
    yaxis = dict(title = 'y', range = [np.min(y), np.max(y)]),
    zaxis = dict(title = 'z', range = [np.min(z), np.max(z)]),
    camera = camera
)

layout_3d_b = go.Layout(
    template = 'plotly_dark',
    title = 'stars in orion',
    scene = scene,
    dragmode = 'turntable'
    )
fig_3d_dust_b = go.Figure(data=[gaia_stars_scatter3D, sun_scatter3D, volume_dust], layout=layout_3d_b)
fig_3d_dust_b.write_html(plot_save_directory + 'orion_3d_dust_centered.html')

### 3D animation

In [23]:
# --------------- Cell block not relevant to Plotly -----------------
def move_clusters(x, y, z, U, V, W, time_steps):
    # get relative, reverse velocities
    U_rel = -(U - np.median(U))
    V_rel = -(V - np.median(V))
    W_rel = -(W - np.median(W))   

    x_positions = np.zeros((len(x), len(time_steps)))
    y_positions = np.zeros((len(y), len(time_steps)))
    z_positions = np.zeros((len(z), len(time_steps)))    
    
    # Calculate the positions for each time step
    for i, t in enumerate(time_steps):
        # Calculate the positions
        # pc/myr to km/s
        x_positions[:, i] = x - U_rel * (t)
        y_positions[:, i] = y - V_rel * (t)
        z_positions[:, i] = z - W_rel * (t)

    return x_positions.T, y_positions.T, z_positions.T

In [24]:
time = np.arange(0, -30, -0.5) # units of Myr
xc, yc, zc, U, V, W = df_clusters[['x_helio', 'y_helio', 'z_helio', 'U', 'V', 'W']].values.T
x_int, y_int, z_int = move_clusters(xc, yc, zc, U, V, W, time)

df_int = pd.DataFrame({'x' : x_int.flatten(), 'y' : y_int.flatten(), 'z' : z_int.flatten(), 'time' : np.repeat(time, len(xc))})
df_int['Group'] = np.tile(df_clusters['Group'].values, len(time))
df_int['Age'] = np.tile(df_clusters['Age'].values, len(time))
df_int['marker_size'] = 20
df_int.loc[(df_int['Age'] - (-1)*df_int['time']) < 0., 'marker_size'] = 0.

In [25]:
frames = []
for t in time:
    df_t = df_int[df_int['time'] == t]
    scatter_clusters_t = go.Scatter3d(
        x = df_t['x'],
        y = df_t['y'],
        z = df_t['z'],
        mode = 'markers',
        marker = dict(
            size = df_t['marker_size'],
            opacity = 1.,
            color = df_t['Age'],
            colorscale = 'plasma',
            line = dict(
                width = 0)
        ),
        name = 'Orion Clusters',
        hovertext = df_t['Group'],
        showlegend = True
    )
    frame = go.Frame(data=[scatter_clusters_t], name=str(t))
    frames.append(frame)

sliders = [dict(
    steps = [dict(
        args = [[f.name], dict(mode='immediate', frame=dict(duration=2, redraw=True))],
        label = str(f.name),
        method = 'animate',
    ) for f in frames],
    active = 0,
    transition = dict(duration=0),
    x = 0,
    y = 0,
)]

play_pause = dict(
    type='buttons',
    showactive=False,
    buttons=[dict(label='Play',
                  method='animate',
                  args=[None, dict(frame=dict(duration=200, redraw=True), fromcurrent=True)]),
             dict(label='Pause',
                  method='animate',
                  args=[[None], dict(frame=dict(duration=0, redraw=True), mode='immediate')])]
)


# plot the clusters' trajectories
cluster_trajectories = go.Scatter3d(
    x = x_int.flatten(),
    y = y_int.flatten(),
    z = z_int.flatten(),
    mode = 'markers',
    marker = dict(
        size = 1,
        opacity = 1.,
        color = df_int['Age'],
        colorscale = 'plasma',
        line = dict(
            width = 0)),
    name = 'Cluster Trajectories',
    visible = 'legendonly'
)




# modifying the layout
layout_3d_clusters_anim = copy.deepcopy(layout_3d_b)
layout_3d_clusters_anim['scene']['xaxis']['range'] = [np.min(x) - 100, np.max(x) + 100]
layout_3d_clusters_anim['scene']['yaxis']['range'] = [np.min(y) - 100, np.max(y) + 100]
layout_3d_clusters_anim['scene']['zaxis']['range'] = [np.min(z) - 100, np.max(z) + 100]
layout_3d_clusters_anim['sliders'] = sliders
layout_3d_clusters_anim['updatemenus'] = [play_pause]



# adding other traces
data = list(frames[0]['data'])
volume_dust['visible'] = 'legendonly'
gaia_stars_scatter3D['visible'] = 'legendonly'
data.append(cluster_trajectories)
data.append(volume_dust)
data.append(gaia_stars_scatter3D)
fig_clusters = go.Figure(data = data, frames=frames, layout=go.Layout(layout_3d_clusters_anim))
fig_clusters.write_html(plot_save_directory + 'orion_3d_clusters_animation.html', auto_open=False, auto_play=False)
#fig_clusters.show()

# Dropdown menus

In [26]:
fig_with_dropdown = go.Figure()

# Add 2D traces
fig_with_dropdown.add_trace(gaia_stars_scatter)
fig_with_dropdown.add_trace(dust_image)

# Add 3D traces
fig_with_dropdown.add_trace(gaia_stars_scatter3D)
fig_with_dropdown.add_trace(volume_dust)

# Add dropdown
# Create a dropdown menu to switch between 2D and 3D plots
fig_with_dropdown.update_layout(
    updatemenus=[
        {
            'buttons': [
                {
                    'args': [
                        {'visible': [True, True, False, False]},  # 2D plot is visible, 3D is not
                        {'xaxis': {'visible': True}, 'yaxis': {'visible': True}},
                        {'scene': {'visible': False}}  # 3D scene is hidden
                    ],
                    'label': '2D',
                    'method': 'update'
                },
                {
                    'args': [
                        {'visible': [False, False, True, True]},  # 2D plot is hidden, 3D is visible
                        {'xaxis': {'visible': False}, 'yaxis': {'visible': False}},
                        {'scene': {'visible': True}}  # 3D scene is visible
                    ],
                    'label': '3D',
                    'method': 'update'
                }
            ],
            'direction': 'down',
            'showactive': True,
            'x': 0.1,
            'y': 1.1,
        }
    ]
)

# Set initial 2D and 3D layout
fig_with_dropdown.update_layout(
    width = 800,
    height = 800,
    template = 'plotly_dark',
    scene=dict(
        aspectmode = 'cube',
        xaxis_title='x (pc)',
        yaxis_title='y (pc)',
        zaxis_title='z (pc)',
        #bgcolor='black'
    ),
    xaxis=dict(title='RA (deg)', visible=True, range = [100, 70]),
    yaxis=dict(title='DEC (deg)', visible=True, range = [-20, 20], scaleanchor = 'x', scaleratio = 1),
)
fig_with_dropdown.write_html(plot_save_directory + 'orion_dropdown_2d_3d.html')

# Plotly Express example

### Before with graph objects (go)

In [27]:

gaia_stars_scatter = go.Scatter(
    x = df_stars['RA_ICRS'],
    y = df_stars['DE_ICRS'],
    mode = 'markers',
    marker = dict(
        size = 2,
        opacity = 1.,
        color = df_stars['Age'], # <---- NOTE: color points by a variable
        colorscale = 'plasma',
    ),
    name = 'Orion Gaia Stars',
    hovertext = df_stars['Name']
)

dust_image = go.Heatmap(
    z = dust_2d,
    x = ra,
    y = dec,
    colorscale = 'inferno',
    zmin = 0,
    zmax = 0.5,
    opacity=1.,
    showscale=False,
    showlegend=True,
    name = 'Dust Map'
)

fig_ra_dec.add_traces([gaia_stars_scatter, dust_image])
layout_ra_dec['annotations'] = None
fig_ra_dec.update_layout(layout_ra_dec)
fig_ra_dec.write_html(plot_save_directory + 'orion_ra_dec.html')

#fig_ra_dec.show()

### With plotly express

In [28]:
df_stars['size'] = .1
scatter_stars = px.scatter(
    df_stars, 
    x='RA_ICRS', 
    y='DE_ICRS', 
    color='Age', 
    hover_name='Group', 
    range_x = [100, 70],
    range_y = [-20, 20],
    height = 800,
    width = 800,
    template = 'plotly_dark'
    )

#scatter_stars.show()